In [ ]:
import selenium
from selenium import webdriver as wb
import pandas as pd
import time
from tqdm import tqdm

In [ ]:
wbD = wb.Chrome('chromedriver.exe')

In [ ]:
wbD.get('https://www.commerciallawpublishers.com/')
publications=wbD.find_element_by_class_name('dropdown-content')
arr=publications.find_elements_by_tag_name('a')
links=[]
cat=[]
for i in arr:
    links.append(i.get_attribute('href'))
    cat.append(i.get_attribute('innerHTML'))

In [ ]:
alldetails=[]
for i in range(len(links)):
    try:
        print("going to publication:",i+1)
        wbD.get(links[i])
        category=cat[i].strip()
        listOflinks =[]
        condition =True
        page=1
        while condition:
            time.sleep(3)
            productInfoList=wbD.find_elements_by_class_name('tab-p-info')
            for el in productInfoList:
                pp2=el.find_element_by_tag_name('a')
                listOflinks.append(pp2.get_property('href'))
            try:
                wbD.find_element_by_link_text("Next").get_property('href')
                wbD.find_element_by_link_text("Next").click()
            except:
                condition=False
        for i in tqdm(listOflinks):
            try:
                wbD.get(i)
                time.sleep(3)
                name = wbD.find_element_by_class_name('mt0').text
                price=wbD.find_element_by_class_name('sp-price').text
                temp ={
                    'Product Name':name,
                    'Category Name':category,
                    'Availability':'-',
                    'Book Format':'-',
                    'Language':'-',
                    'Writer':'-',
                    'ISBN':'-',
                    'EDITION':'-',
                    'PRODUCTION YEAR':'-',
                    'price':price,
                    'linkofproduct':i
                }
                hello=wbD.find_element_by_class_name('single-product-info').find_elements_by_tag_name('p')
                for i in hello:
                    try:
                        column=i.find_element_by_tag_name('b').text
                        value=i.find_element_by_class_name('color1').text
                        temp[column[:len(column)-1]]=value
                    except:
                        continue
               # print(temp)
                alldetails.append(temp)
            except:
                continue
    except:
        continue
        
data = pd.DataFrame(alldetails)  

In [ ]:
from_6=data.copy()
data=from_6[['price']].copy()
data['price'] = data['price'].astype('str')
lion=data['price'].str.split('/-')
main_price=[]
offer=[]
old_price=[]
for p in lion:
    if len(p)==3:
        main_price.append(p[0].strip())
        old_price.append(p[1].strip())
        offer.append(p[2].strip())
    else:
        main_price.append(p[0].strip())
        old_price.append('-')
        offer.append('-')
from_6['main_price']=main_price
from_6['old_price']=old_price
from_6['offer']=offer
from_6.drop(['price'], axis = 1,inplace=True)

In [ ]:
from_6.to_excel('final.xlsx')